In [1]:
library(rCharts)

In [ ]:
d <- expand.grid(g = letters[1:4], x = 1:12)
d$y <- rpois(nrow(d), 100)
p <- hPlot(y~x, group = "g", data = d)
p$show()

In [ ]:
xdata <- c(1, 2, 3, 4, 5)
ydata <- c(2, 3, 4, 1, 5)
data <- data.frame(x = xdata, y = ydata)
print(data)
p <- hPlot(y~x, data=data)
p$show()

In [2]:
model2pl <- function(x, a, b) {
    return (1.0 / (1.0 + exp(-1.701 * a * (x - b))))
}

In [12]:
xdata <- seq(-4.0, 4.01, by=0.1)
ydata1 <- model2pl(xdata, a=0.5, b=-1.0)
ydata2 <- model2pl(xdata, a=1.0, b=-0.5)
ydata3 <- model2pl(xdata, a=1.5, b=0.0)
ydata4 <- model2pl(xdata, a=2.0, b=0.5)
ydata5 <- model2pl(xdata, a=3.0, b=1.0)

In [21]:

data1 <- data.frame(type=rep('A', length=81), x=xdata, y=ydata1)
data2 <- data.frame(type=rep('B', length=81), x=xdata, y=ydata2)
data3 <- data.frame(type=rep('C', length=81), x=xdata, y=ydata3)
data4 <- data.frame(type=rep('D', length=81), x=xdata, y=ydata4)
data5 <- data.frame(type=rep('E', length=81), x=xdata, y=ydata5)

data <- rbind(data1, data2, data3, data4, data5)

In [35]:
p <- hPlot(y~x, group="type", data=data, type = 'line', title = '項目特性曲線', subtitle = 'Item Characteristic Curves')
p$chart(zoomType = "xy")
p$exporting(enabled = T)
p$xAxis(title = list(text = "Ability"), min = -4.0, max = 4.0, gridLineWidth = 1)
p$yAxis(title = list(text = "Probability"), min = 0.0, max = 1.0, gridLineWidth = 1)
p$plotOptions(line = list(marker = list(enabled = F)))
p$legend(align = 'right', verticalAlign = 'bottom', layout = 'vertical')
p$show()

In [10]:
p <- hPlot(ydata~xdata, data)
p$show()

In [ ]:
xdata <- seq(-4.0, 4.01, by =0.1)
print(xdata)
ydata <- apply(xdata, 1, model2pl)
print(ydata)

In [36]:
library(ltm)

Loading required package: MASS
Loading required package: msm
Loading required package: polycor
Loading required package: mvtnorm
Loading required package: sfsmisc


In [37]:
csvdata <- read.csv('~/IrtWeb/irt_sample.csv', header = TRUE)

In [39]:
result <- ltm(csvdata~z1, IRT.param = TRUE)

In [55]:
coef0 <- result$coefficient
coef0

,(Intercept),z1
A,1.430019,1.215814
B,0.6164858,1.5742688
C,0.8940602,16.8993942
D,-0.2354762,1.1696718
E,-1.368762,1.181162


In [56]:
attributes(coef0)

$dim
[1] 5 2

$dimnames
$dimnames[[1]]
[1] "A" "B" "C" "D" "E"

$dimnames[[2]]
[1] "(Intercept)" "z1"

In [71]:
dimnames(coef0)

[[1]]
[1] "A" "B" "C" "D" "E"

[[2]]
[1] "(Intercept)" "z1"

In [93]:
coef0[2,2]

[1] 1.574269

In [80]:
dimnames(coef0)[[1]][1]

[1] "A"

In [84]:
valsize = length(dimnames(coef0)[[1]])
valsize

[1] 5

In [94]:
data <- data.frame()
xdata <- seq(-4.0, 4.01, by=0.1)
for (i in 1:valsize) {
    ydata_tmp <- model2pl(xdata, a=coef0[i, 2], b=coef0[i, 1])
    data_tmp <- data.frame(type=rep(dimnames(coef0)[[1]][i], length=81), x=xdata, y=ydata_tmp)
    data <- rbind(data, data_tmp)
}

In [ ]:
data

In [96]:
p <- hPlot(y~x, group="type", data=data, type = 'line', title = '項目特性曲線', subtitle = 'Item Characteristic Curves')
p$chart(zoomType = "xy")
p$exporting(enabled = T)
p$xAxis(title = list(text = "Ability"), min = -4.0, max = 4.0, gridLineWidth = 1)
p$yAxis(title = list(text = "Probability"), min = 0.0, max = 1.0, gridLineWidth = 1)
p$plotOptions(line = list(marker = list(enabled = F)))
p$legend(align = 'right', verticalAlign = 'bottom', layout = 'vertical')
p$show()

In [98]:
result


Call:
ltm(formula = csvdata ~ z1, IRT.param = TRUE)

Coefficients:
   Dffclt  Dscrmn
A  -1.176   1.216
B  -0.392   1.574
C  -0.053  16.899
D   0.201   1.170
E   1.159   1.181

Log.Lik: -56.733


In [102]:
summary <- summary(result)

In [103]:
str(summary)

List of 11
 $ coefficients: num [1:10, 1:3] -1.1762 -0.3916 -0.0529 0.2013 1.1588 ...
  ..- attr(*, "dimnames")=List of 2
  .. ..$ : chr [1:10] "Dffclt.A" "Dffclt.B" "Dffclt.C" "Dffclt.D" ...
  .. ..$ : chr [1:3] "value" "std.err" "z.vals"
 $ Var.betas   : num [1:10, 1:10] 0.5771 0.1069 0.5306 0.0815 0.0997 ...
  ..- attr(*, "dimnames")=List of 2
  .. ..$ : chr [1:10] "(Intercept).1" "(Intercept).2" "(Intercept).3" "(Intercept).4" ...
  .. ..$ : chr [1:10] "(Intercept).1" "(Intercept).2" "(Intercept).3" "(Intercept).4" ...
 $ logLik      : num -56.7
 $ AIC         : num 133
 $ BIC         : num 143
 $ max.sc      : num 0.000346
 $ conv        : int 0
 $ counts      : Named int [1:2] 2 1
  ..- attr(*, "names")= chr [1:2] "function" "gradient"
 $ call        : language ltm(formula = csvdata ~ z1, IRT.param = TRUE)
 $ ltst        :List of 5
  ..$ factors: int 1
  ..$ inter  : logi FALSE
  ..$ quad.z1: logi FALSE
  ..$ quad.z2: logi FALSE
  ..$ nams   : chr [1:2] "(Intercept)" "z1"
 $ cont

In [104]:
summary$coefficients

,value,std.err,z.vals
Dffclt.A,-1.1761828,0.7697059,-1.5280937
Dffclt.B,-0.3916013,0.4449440,-0.8801138
Dffclt.C,-0.05290487,0.82343727,-0.06424881
Dffclt.D,0.2013182,0.5043902,0.3991319
Dffclt.E,1.1588266,0.8257548,1.4033543
Dscrmn.A,1.2158138,0.9135998,1.3307947
Dscrmn.B,1.574269,1.045622,1.505581
Dscrmn.C,16.89939422,260.58495015,0.06485177
Dscrmn.D,1.1696718,0.8445942,1.3848921
Dscrmn.E,1.1811624,0.9752508,1.2111371


In [105]:
class(summary$coefficients)

[1] "matrix"

In [128]:
coef <- summary$coefficients[,1]

In [131]:
coef[0]

named numeric(0)

In [108]:
result$coefficients

,(Intercept),z1
A,1.430019,1.215814
B,0.6164858,1.5742688
C,0.8940602,16.8993942
D,-0.2354762,1.1696718
E,-1.368762,1.181162


In [109]:
result


Call:
ltm(formula = csvdata ~ z1, IRT.param = TRUE)

Coefficients:
   Dffclt  Dscrmn
A  -1.176   1.216
B  -0.392   1.574
C  -0.053  16.899
D   0.201   1.170
E   1.159   1.181

Log.Lik: -56.733


In [110]:
class(result)

[1] "ltm"

In [111]:
-1.701*1.215814*1.176

[1] -2.432085

In [112]:
1/(1+2.718281828^(-2.432085146))

[1] 0.9192415

In [114]:
str(result)


List of 14
 $ coefficients: num [1:5, 1:2] 1.43 0.616 0.894 -0.235 -1.369 ...
  ..- attr(*, "dimnames")=List of 2
  .. ..$ : chr [1:5] "A" "B" "C" "D" ...
  .. ..$ : chr [1:2] "(Intercept)" "z1"
 $ log.Lik     : num -56.7
 $ convergence : int 0
 $ hessian     : num [1:10, 1:10] 2.786 -0.247 -0.232 -0.193 -0.134 ...
 $ counts      : Named int [1:2] 2 1
  ..- attr(*, "names")= chr [1:2] "function" "gradient"
 $ patterns    :List of 2
  ..$ X  : num [1:12, 1:5] 0 0 0 0 1 1 1 1 1 1 ...
  ..$ obs: int [1:12] 2 1 1 1 3 1 1 2 2 1 ...
 $ GH          :List of 2
  ..$ Z  : num [1:21, 1:2] 1 1 1 1 1 1 1 1 1 1 ...
  ..$ GHw: num [1:21] 2.10e-14 4.98e-11 1.45e-08 1.23e-06 4.22e-05 ...
 $ max.sc      : num 0.000346
 $ ltst        :List of 5
  ..$ factors: int 1
  ..$ inter  : logi FALSE
  ..$ quad.z1: logi FALSE
  ..$ quad.z2: logi FALSE
  ..$ nams   : chr [1:2] "(Intercept)" "z1"
 $ X           :'data.frame':	20 obs. of  5 variables:
  ..$ A: int [1:20] 0 0 0 1 1 1 1 1 1 0 ...
  ..$ B: int [1:20] 0

In [118]:
result$coefficients

,(Intercept),z1
A,1.430019,1.215814
B,0.6164858,1.5742688
C,0.8940602,16.8993942
D,-0.2354762,1.1696718
E,-1.368762,1.181162


In [144]:
library(rCharts)
library(ltm)

In [145]:
csvdata <- read.csv('~/IrtWeb/irt_sample.csv', header = TRUE)

In [132]:
result <- ltm(csvdata~z1, IRT.param = TRUE)

In [133]:
coef0 <- result$coefficient
coef1 <- summary(result)$coefficients

In [136]:
valsize = length(dimnames(coef0)[[1]])

In [143]:
model2pl <- function(x, a, b) {
    return (1.0 / (1.0 + exp(-1.701 * a * (x - b))))
}

In [141]:
data <- data.frame()
xdata <- seq(-4.0, 4.01, by=0.1)
for (i in 1:valsize) {
    ydata_tmp <- model2pl(xdata, a=coef0[i, 2], b=coef1[i, 1])
    data_tmp <- data.frame(type=rep(dimnames(coef0)[[1]][i], length=81), x=xdata, y=ydata_tmp)
    data <- rbind(data, data_tmp)
}

In [142]:
p <- hPlot(y~x, group="type", data=data, type = 'line', title = '項目特性曲線', subtitle = 'Item Characteristic Curves')
p$chart(zoomType = "xy")
p$exporting(enabled = T)
p$xAxis(title = list(text = "Ability"), min = -4.0, max = 4.0, gridLineWidth = 1)
p$yAxis(title = list(text = "Probability"), min = 0.0, max = 1.0, gridLineWidth = 1)
p$plotOptions(line = list(marker = list(enabled = F)))
p$legend(align = 'right', verticalAlign = 'bottom', layout = 'vertical')
p$show()

rCharts 利用の際にはライブラリ読み込みを行う。

In [18]:
require("rCharts")

sin曲線と cos曲線をグラフに表示することを目標とする。
それぞれのデータを作成するために、まず x 座標の数列を作成して変数 xdata に入れるとともに、
その個数を変数 nsize に入れる。

In [19]:
xdata <- seq(-1.0, 7.01, by=0.1)
nsize <- length(xdata)

x の値を示す数列データ xdata のそれぞれの値に対応する sin(x) の値でできた数列データを sindata, xdata のそれぞれの値に対応する cos(x) の値でできた数列データを cosdata とする。

In [20]:
sindata <- sin(xdata)
cosdata <- cos(xdata)

sin曲線上の (x, y) の値 nsize 組、cos曲線上の (x, y) の値 nsize 組、そして、それぞれの前に sin データか cos データかを示すラベルをつけ、結合させたデータフレームを作成する。

| type  |  x  |  y  |
| ----  | ---- | ----- |
| "sin" | -1.0 | -0.345 |
|   :   |   :   |  :  |
| "cos" |   0   |  1  |
|   :   |   :   |  :  |


In [21]:
data <- rbind( data.frame(type=rep("sin", nsize), x = xdata, y = sindata),
              data.frame(type=rep("cos", nsize), x = xdata, y = cosdata) )

データフレーム data の x, y を、HighCharts によってグラフ化する。
各種の詳細設定を指定は、HighCharts での Javascript の形式を下記のように書き換えて指定する。

In [26]:
p <- hPlot(y~x, group="type", data=data, type = 'line', title = '三角関数', subtitle = 'rCharts/HighCharts Demo')
p$chart(zoomType = "xy")
p$exporting(enabled = T)
p$xAxis(title = list(text = "x"), min = -1.0, max = 7.0, gridLineWidth = 1)
p$yAxis(title = list(text = "y = sin(x), y = cos(x)"), min = -1.2, max = 1.2, gridLineWidth = 1)
p$plotOptions(line = list(marker = list(enabled = F), enableMouseTracking = F))
p$legend(align = 'right', verticalAlign = 'middle', layout = 'vertical')
p$show()